In [74]:
"""
    first-cut: RandomForest on clean columns
"""

'\n    first-cut: RandomForest on clean columns\n'

In [78]:
def select_columns(df):
    columns = df.columns.tolist()
    columns.remove('PassengerId')
    columns.remove('Name')
    columns.remove('Age')
    columns.remove('Cabin')
    columns.remove('Ticket')
    return columns

def clean(df):
    """
        select desired columns and drop the rest
    """
    columns = select_columns(df)
    df = df[columns]
    
    """
         imputation: replace/drop na values
    """
    df['Embarked'].fillna('S', inplace=True)
    
    return df

In [79]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def get_features(df):
    features = df.columns.tolist()
    features.remove('Survived')
    
    categorical_features = [feature for feature in features]
    categorical_features.remove('Fare')
    
    numerical_features = ['Fare']
    return categorical_features, numerical_features

def encode(df_cat):
    return df_cat.apply(LabelEncoder().fit_transform)

def get_feature_vector(df):
    categorical_features, numerical_features = get_features(df)
    
    X_cat = encode(df[categorical_features])
    X_num = df[numerical_features]
    
    X = pd.concat([X_cat, X_num], axis=1)
    
    return X

def get_label_vector(df):
    y = df['Survived']
    return y

In [102]:
import pandas as pd

df = pd.read_csv('../Data/titanic/train.csv')

df = clean(df)

X = get_feature_vector(df)
y = get_label_vector(df)

X_train, X_test, y_train, y_test = train_test_split(X, y)

from sklearn.model_selection import cross_val_score
def train(model, X_train, y_train):
    model.fit(X_train, y_train)
    print("cross-validation scores: {}".format(cross_val_score(rf, X_train, y_train, cv=5)))
    return model

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
def test(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print("accuracy: {}".format(accuracy_score(y_pred, y_test)))
    print(pd.DataFrame(confusion_matrix(y_pred, y_test), 
             index=['true died', 'true survived'], 
             columns=['pred died', 'pred survied']))
    
   

/home/raghav/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [103]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=4, min_samples_leaf=10, n_estimators=40)
train(rf, X_train, y_train)
test(rf, X_test, y_test)

cross-validation scores: [0.76865672 0.8358209  0.80597015 0.80597015 0.81818182]
accuracy: 0.8071748878923767
               pred died  pred survied
true died            124            37
true survived          6            56
